## LightGBM-Optunaのサンプルコード（train, valid, test使用）

In [1]:
%load_ext lab_black

In [2]:
# ライブラリーのインポート
import os

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

# ボストンの住宅価格データ
from sklearn.datasets import load_boston

# 前処理
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# LightGBM
import lightgbm as lgb

# Optuna
import optuna
from optuna.samplers import CmaEsSampler

# 評価指標
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [3]:
print(lgb.__version__)

3.1.1


In [4]:
print(optuna.__version__)

2.8.0


In [5]:
# データセットの読込み
boston = load_boston()

# 説明変数の格納
df = pd.DataFrame(boston.data, columns=boston.feature_names)
# 目的変数の追加
df["MEDV"] = boston.target

# データの中身を確認
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


#### 前処理

In [6]:
# ランダムシード値
RANDOM_STATE = 10

# 学習データと評価データの割合
TEST_SIZE = 0.2

# 学習データと評価データを作成
x_train, x_test, y_train, y_test = train_test_split(
    df.iloc[:, 0 : df.shape[1] - 1],
    df.iloc[:, df.shape[1] - 1],
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
)

# trainのデータセットの2割をモデル学習時のバリデーションデータとして利用する
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train, y_train, test_size=TEST_SIZE, random_state=RANDOM_STATE
)

In [7]:
def objective(trial):

    params = {
        "objective": "mean_squared_error",
        "metric": "rmse",
        "lambda_l1": trial.suggest_loguniform("lambda_l1", 1e-8, 10.0),
        "lambda_l2": trial.suggest_loguniform("lambda_l2", 1e-8, 10.0),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_uniform("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_uniform("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
    }

    # LightGBMを利用するのに必要なフォーマットに変換
    lgb_train = lgb.Dataset(x_train, y_train)
    lgb_eval = lgb.Dataset(x_valid, y_valid, reference=lgb_train)

    # 学習の経過を保存する箱
    evaluation_results = {}

    model = lgb.train(
        params,
        lgb_train,
        num_boost_round=200,
        valid_names=["train", "valid"],
        valid_sets=[lgb_train, lgb_eval],
        evals_result=evaluation_results,
    )

    preds = model.predict(x_valid)
    mae = mean_absolute_error(y_valid, preds)

    return mae

In [8]:
%%time
# optunaで最適値を見つける
# create_studyメソッドの引数"sampler"にサンプラーと乱数シードを指定
study = optuna.create_study(direction='minimize', sampler=CmaEsSampler(seed=RANDOM_STATE))
study.optimize(objective, n_trials=50)

[I 2021-07-08 06:39:09,145] A new study created in memory with name: no-name-42e32b63-ff35-4ebe-9044-3cb18711dda2


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000123 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 807
[LightGBM] [Info] Number of data points in the train set: 323, number of used features: 12
[LightGBM] [Info] Start training from score 21.968731
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's rmse: 8.49905	valid's rmse: 6.64307
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's rmse: 7.94829	valid's rmse: 6.23289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's rmse: 7.41287	valid's rmse: 5.80693
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's rmse: 6.92764	valid's rmse: 5.45293
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's rmse: 6.55394	valid's rmse: 5.10492
[LightGBM] [Warning] No further splits wit

[I 2021-07-08 06:39:09,426] Trial 0 finished with value: 2.5374505963248395 and parameters: {'lambda_l1': 0.08747537025773001, 'lambda_l2': 1.537331564587801e-08, 'num_leaves': 163, 'feature_fraction': 0.8492823295231671, 'bagging_fraction': 0.6991042073815543, 'bagging_freq': 2, 'min_child_samples': 24}. Best is trial 0 with value: 2.5374505963248395.
[I 2021-07-08 06:39:09,565] Trial 1 finished with value: 2.878060174783933 and parameters: {'lambda_l1': 0.00032367457948696284, 'lambda_l2': 0.0003524580750386093, 'num_leaves': 129, 'feature_fraction': 0.590365804096321, 'bagging_fraction': 0.6461745560080591, 'bagging_freq': 4, 'min_child_samples': 52}. Best is trial 0 with value: 2.5374505963248395.


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 807
[LightGBM] [Info] Number of data points in the train set: 323, number of used features: 12
[LightGBM] [Info] Start training from score 21.968731
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[1]	train's rmse: 8.62649	valid's rmse: 6.65379
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[2]	train's rmse: 8.40896	valid's rmse: 6.42121
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[3]	train's rmse: 7.99417	valid's rmse: 6.04419
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[4]	train's rmse: 7.63313	valid's rmse: 5.75138
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[5]	train's rmse: 7.42911	valid's rmse: 5.57125
[LightGBM] [Warning] No further splits wit

[I 2021-07-08 06:39:09,725] Trial 2 finished with value: 2.973989150517948 and parameters: {'lambda_l1': 0.000323226468873299, 'lambda_l2': 0.00034492480918485257, 'num_leaves': 129, 'feature_fraction': 0.7389068226679032, 'bagging_fraction': 0.6346035549247403, 'bagging_freq': 4, 'min_child_samples': 53}. Best is trial 0 with value: 2.5374505963248395.


...

No further splits with positive gain, best gain: -inf
[189]	train's rmse: 3.07979	valid's rmse: 3.87352
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[190]	train's rmse: 3.07724	valid's rmse: 3.86328
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[191]	train's rmse: 3.07312	valid's rmse: 3.85999
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[192]	train's rmse: 3.06868	valid's rmse: 3.85774
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[193]	train's rmse: 3.06414	valid's rmse: 3.85527
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[194]	train's rmse: 3.05892	valid's rmse: 3.84786
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[195]	train's rmse: 3.05232	valid's rmse: 3.83575
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[196]	train's rmse: 3.04706	valid's rmse: 3.82534
[LightGBM] [Warning] 

[I 2021-07-08 06:39:18,262] Trial 48 finished with value: 2.6581628684810683 and parameters: {'lambda_l1': 0.000312188341024563, 'lambda_l2': 0.00018266117766057197, 'num_leaves': 129, 'feature_fraction': 0.6461688404265603, 'bagging_fraction': 0.9865153961010585, 'bagging_freq': 4, 'min_child_samples': 52}. Best is trial 0 with value: 2.5374505963248395.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[134]	train's rmse: 3.13207	valid's rmse: 3.87037
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[135]	train's rmse: 3.12408	valid's rmse: 3.86111
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[136]	train's rmse: 3.11985	valid's rmse: 3.84821
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[137]	train's rmse: 3.11621	valid's rmse: 3.84412
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[138]	train's rmse: 3.11135	valid's rmse: 3.83632
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[139]	train's rmse: 3.10715	valid's rmse: 3.83879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[140]	train's rmse: 3.10182	valid's rmse: 3.84298
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[141]	train's rmse: 3.09804	valid's rmse: 3.83649


[I 2021-07-08 06:39:18,493] Trial 49 finished with value: 2.731340803789171 and parameters: {'lambda_l1': 0.00028256707403869575, 'lambda_l2': 0.0001974825906968447, 'num_leaves': 129, 'feature_fraction': 0.7190147104794153, 'bagging_fraction': 0.9178590361493344, 'bagging_freq': 4, 'min_child_samples': 52}. Best is trial 0 with value: 2.5374505963248395.



[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[74]	train's rmse: 3.67179	valid's rmse: 3.89631
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[75]	train's rmse: 3.65394	valid's rmse: 3.87432
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[76]	train's rmse: 3.64786	valid's rmse: 3.87929
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[77]	train's rmse: 3.63768	valid's rmse: 3.89348
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[78]	train's rmse: 3.6229	valid's rmse: 3.92566
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[79]	train's rmse: 3.61326	valid's rmse: 3.92644
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[80]	train's rmse: 3.60434	valid's rmse: 3.92167
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[81]	train's rmse: 3.59243	valid's rmse: 3.92843
[LightGB

In [9]:
best_params = study.best_params
best_params["random_state"] = RANDOM_STATE
best_params

{'lambda_l1': 0.08747537025773001,
 'lambda_l2': 1.537331564587801e-08,
 'num_leaves': 163,
 'feature_fraction': 0.8492823295231671,
 'bagging_fraction': 0.6991042073815543,
 'bagging_freq': 2,
 'min_child_samples': 24,
 'random_state': 10}

In [10]:
# LightGBMを利用するのに必要なフォーマットに変換
lgb_train = lgb.Dataset(x_train, y_train)

# チューニングしたハイパーパラメーターをフィット
optimised_model = lgb.train(best_params, lgb_train)

# LightGBM推論
y_pred = optimised_model.predict(x_test)

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000108 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 807
[LightGBM] [Info] Number of data points in the train set: 323, number of used features: 12
[LightGBM] [Info] Start training from score 21.968731
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain

In [11]:
# 評価
def calculate_scores(true, pred):
    """全ての評価指標を計算する

    Parameters
    ----------
    true (np.array)       : 実測値
    pred (np.array)       : 予測値

    Returns
    -------
    scores (pd.DataFrame) : 各評価指標を纏めた結果

    """
    scores = {}
    scores = pd.DataFrame(
        {
            "R2": r2_score(true, pred),
            "MAE": mean_absolute_error(true, pred),
            "MSE": mean_squared_error(true, pred),
            "RMSE": np.sqrt(mean_squared_error(true, pred)),
        },
        index=["scores"],
    )
    return scores

In [12]:
scores = calculate_scores(y_test, y_pred)
print(scores)

              R2       MAE        MSE      RMSE
scores  0.804698  2.999387  20.424788  4.519379
